# Prepare Sales Price Data
I want to study sales price data and develop a model that can predict the sales price of a home in Fairfax County, VA, based on its characteristics. To do this a dataset containing sales prices and home characteristics is needed, in this notebook datasets collected from different sources are combined to produce such a dataset.

**Data Sources**

- Fairfax County provides a variety of data through its open geospatial data site: https://data-fairfaxcountygis.opendata.arcgis.com/. The datasets below were downloaded from this portal and saved as csv files.
    - Real Estate Sales data, contains sales price and date, as well as type of sale
    - Parcels data, contains general information about the parcel and zoning
    - Dwelling data, contains general information about the structure on the parcel, e.g. year built, rooms, sq ft, etc.
    - Address data, contains location information about the parcel, including address but also supervisor district and x/y coordinates in NAD83 (EPSG:2283) format
- The Washington Metropolitan Area Transit Authority (WMATA) provides free APIs that can be used to get a variety of information describing bus and rail transit in the region. I used it to get very general rail station information, I'm mainly interested in using the location of rail stations as an additional feature in predicting home sales prices.

**Caveats**

To keep things simple I want to start with 2023 single family homes sales and see how well a model can predict the 2023 sales price of a home given its characteristics. Obviously in the medium/long term home prices change, so a model trained on one year of sales data would become less useful over time. As a future extension it might be interesting to find a way to adjust for this. One idea might be to create a previous-month/previous-year median sales price metric and use that as an input, allowing the model to adjust it's prediction based on how the overall market differs from the training data. Another thing to note is the scope of this model will be limited to single family homes. Fairfax County has a variety of housing types including significant numbers of townhomes and condominiums, adjustments or separate models for these property types could be an interesting extension as well.

**Data Preparation**

The Fairfax County data uses the parcel id as a common key across many of its real estate datasets, so I will use this to join parcel, dwelling, and address data to 2023 single family home sales. As additional features beyond what's available in the Fairfax County data, I want to calculate the distance from sold homes to different metro stations and identify the distance to the closest metro station as an additional feature for use in the modeling effort.

In [117]:
import pandas as pd
import datetime as dt
import pyproj

Begin by reading in and combining the different datasets pulled from Fairfax County that provide sales, parcel, dwelling, and location data.

In [16]:
# Read in data
sales_df = pd.read_csv('data/sales_data.csv')
sales_df = sales_df[sales_df['SALEVAL_DESC'] == 'Valid and verified sale']
sales_df['SALEDT'] = pd.to_datetime(sales_df['SALEDT']).dt.tz_localize(None) # set timezone to None, no need to deal with timezones in this context
sales_df = sales_df[sales_df['SALEDT'] >= dt.datetime(1970, 1, 1)]
sales_df[sales_df['SALEDT'].dt.year == 2023]

parcels_df = pd.read_csv('data/parcels_data.csv')[['PARID', 'LIVUNIT', 'LUC_DESC']]
parcels_df['LUC_DESC'] = parcels_df['LUC_DESC'].str.strip()

dwelling_df = pd.read_csv("data/dwelling_data.csv")[['PARID', "YRBLT", "YRREMOD", "RMBED", "FIXBATH", "FIXHALF", "SFLA", "HEAT_DESC", "CDU_DESC", "EXTWALL_DESC", "GRADE_DESC"]]

address_df = pd.read_csv("data/address_data.csv")[['Parcel Identification Number', 'City', 'Supervisor District', 'Address Location X', 'Address Location Y']]
address_df = (
    address_df
    .drop_duplicates(subset=['Parcel Identification Number', 'City', 'Supervisor District'])
    .rename(columns={'Parcel Identification Number':'PARID'})
)

C:\Users\felix\AppData\Local\Temp\ipykernel_20172\2850432412.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_df = pd.read_csv('data/sales_data.csv')
C:\Users\felix\AppData\Local\Temp\ipykernel_20172\2850432412.py:13: DtypeWarning: Columns (8,13,14,24,25,26,37) have mixed types. Specify dtype option on import or set low_memory=False.
  address_df = pd.read_csv("data/address_data.csv")[['Parcel Identification Number', 'City', 'Supervisor District', 'Address Location X', 'Address Location Y']]


In [224]:
prepared_df = (
    sales_df[sales_df['SALEDT'].dt.year == 2023][[
        "PARID", "SALEDT", "PRICE"
    ]].merge(
        dwelling_df,
        on='PARID', 
        how='left'
    )
    .merge(
        address_df,
        on=['PARID'],
        how='left'
    )
    .merge(
        parcels_df,
        on=['PARID'],
        how='left'
    )
)

# Limit to the main property types for residential real estate
prepared_df = prepared_df[prepared_df["LUC_DESC"].isin([
    'Single-family, Detached', 
    'Townhouse in ownership development',
    'Garden Apartments condominium (=<4story)',
    'Multiplex in condominium development',
    'High rise apartments condo(=>9  comm)',
    'High rise apartments condo(=>9 no comm)',
    'Medium rise apartments condo(5to8 stry)',
    'Townhouse in condominium development',
    'Duplex, either vertical or horizontal'
])]

Let's do some quick checks to make sure the data looks as expected, we want to make sure that there aren't duplicate records or records with data quality problems that could get in the way of modeling later on.

In [225]:
# check if there are parcels that are entered for a sale twice on the same date
len(prepared_df) == len(prepared_df[['PARID', 'SALEDT']].drop_duplicates())

False

False here means there are some PARIDs with multiple entries with the same saledt, that seems like a data quality issue.

In [226]:
parid_saledt_counts = prepared_df.drop_duplicates()[['PARID', 'SALEDT']].value_counts().reset_index()
# we just need to check the ones that appear multiple times
parid_saledt_counts = parid_saledt_counts[parid_saledt_counts['count'] > 1]

In [227]:
parid_saledt_counts


,PARID,SALEDT,count
0,0203 08 0005,2023-04-28,2
1,0851 02 0025A,2023-09-28,2
2,0773 10 C2,2023-11-28,2


Since there are only three duplicates we can manually inspect all of them quickly.

The first two look like duplicate entries exist for the same parcel in the dwelling data. Since there are only two of these in this dataset it seem reasonable just to exclude them. If there were many duplicates then it would be worth checking that the underlying dwelling data is formatted as expected and that joining it with other dataframes had the intended outcome.

The third one looks like there were two sales of the same parcel on the same day with different prices. This is also a bit bizarre but since there's only one instance of this I'm comfortable just discarding it.

In [228]:
# since there are only three duplicates we can manually inspect all of them quickly
i=0
while i < len(parid_saledt_counts):
    display(prepared_df[prepared_df['PARID'] == parid_saledt_counts.iloc[i]['PARID']].drop_duplicates())
    display(dwelling_df[dwelling_df['PARID'] == parid_saledt_counts.iloc[i]['PARID']].drop_duplicates())
    print('---')
    i+=1

,PARID,SALEDT,PRICE,YRBLT,YRREMOD,RMBED,FIXBATH,FIXHALF,SFLA,HEAT_DESC,CDU_DESC,EXTWALL_DESC,GRADE_DESC,City,Supervisor District,Address Location X,Address Location Y,LIVUNIT,LUC_DESC
179,0203 08 0005,2023-04-28,1299999.0,1960.0,NaN,5.0,3.0,NaN,1984.0,Central A/C,Poor,Brick,Average 10,MCLEAN,DRANESVILLE,1.184375e+07,7.032645e+06,1.0,"Single-family, Detached"
1447,0203 08 0005,2023-04-28,1250000.0,1960.0,NaN,5.0,3.0,NaN,1984.0,Central A/C,Poor,Brick,Average 10,MCLEAN,DRANESVILLE,1.184375e+07,7.032645e+06,1.0,"Single-family, Detached"


,PARID,YRBLT,YRREMOD,RMBED,FIXBATH,FIXHALF,SFLA,HEAT_DESC,CDU_DESC,EXTWALL_DESC,GRADE_DESC
274877,0203 08 0005,1960.0,NaN,5.0,3.0,NaN,1984.0,Central A/C,Poor,Brick,Average 10


---


,PARID,SALEDT,PRICE,YRBLT,YRREMOD,RMBED,FIXBATH,FIXHALF,SFLA,HEAT_DESC,CDU_DESC,EXTWALL_DESC,GRADE_DESC,City,Supervisor District,Address Location X,Address Location Y,LIVUNIT,LUC_DESC
11285,0851 02 0025A,2023-09-28,1290000.0,2002.0,NaN,1.0,1.0,NaN,732.0,Non Central,NaN,Wood,Average,CLIFTON,SPRINGFIELD,1.179565e+07,6.965997e+06,1.0,"Single-family, Detached"
11286,0851 02 0025A,2023-09-28,1290000.0,1989.0,NaN,4.0,3.0,1.0,3432.0,Central A/C,Average,Wood,Excellent,CLIFTON,SPRINGFIELD,1.179565e+07,6.965997e+06,1.0,"Single-family, Detached"


,PARID,YRBLT,YRREMOD,RMBED,FIXBATH,FIXHALF,SFLA,HEAT_DESC,CDU_DESC,EXTWALL_DESC,GRADE_DESC
55319,0851 02 0025A,2002.0,NaN,1.0,1.0,NaN,732.0,Non Central,NaN,Wood,Average
55320,0851 02 0025A,1989.0,NaN,4.0,3.0,1.0,3432.0,Central A/C,Average,Wood,Excellent


---


,PARID,SALEDT,PRICE,YRBLT,YRREMOD,RMBED,FIXBATH,FIXHALF,SFLA,HEAT_DESC,CDU_DESC,EXTWALL_DESC,GRADE_DESC,City,Supervisor District,Address Location X,Address Location Y,LIVUNIT,LUC_DESC
2410,0773 10 C2,2023-11-28,1875000.0,2006.0,NaN,5.0,7.0,1.0,6488.0,Central A/C,Average,Stucco,Excellent 10,FAIRFAX STATION,SPRINGFIELD,1.181729e+07,6.970576e+06,1.0,"Single-family, Detached"
2411,0773 10 C2,2023-11-28,1875000.0,2012.0,NaN,1.0,1.0,NaN,864.0,Central A/C,Average,Stucco,Average,FAIRFAX STATION,SPRINGFIELD,1.181729e+07,6.970576e+06,1.0,"Single-family, Detached"


,PARID,YRBLT,YRREMOD,RMBED,FIXBATH,FIXHALF,SFLA,HEAT_DESC,CDU_DESC,EXTWALL_DESC,GRADE_DESC
20186,0773 10 C2,2006.0,NaN,5.0,7.0,1.0,6488.0,Central A/C,Average,Stucco,Excellent 10
20187,0773 10 C2,2012.0,NaN,1.0,1.0,NaN,864.0,Central A/C,Average,Stucco,Average


---


In [229]:
# remove the duplicate entries
prepared_df = prepared_df[~ prepared_df['PARID'].isin(parid_saledt_counts['PARID'].tolist())]

Check for entries where any of the most important features are missing. It looks like there are 42 entries missing at least one of the main features we want to use for modeling later on. Since it's a small number of entries relative to the overall size of the dataset it seems reasonable to just remove these records from the prepared dataset as well.

In [230]:
missing_key_features = prepared_df[
    prepared_df['PRICE'].isna()
    | prepared_df['YRBLT'].isna() 
    | prepared_df['RMBED'].isna()
    | prepared_df['FIXBATH'].isna()
    | prepared_df['SFLA'].isna()
    | prepared_df['Supervisor District'].isna()
]
print(len(missing_key_features))
display(missing_key_features.head(5))

42


,PARID,SALEDT,PRICE,YRBLT,YRREMOD,RMBED,FIXBATH,FIXHALF,SFLA,HEAT_DESC,CDU_DESC,EXTWALL_DESC,GRADE_DESC,City,Supervisor District,Address Location X,Address Location Y,LIVUNIT,LUC_DESC
111,0813 13N 0637,2023-11-08,700000.0,1964.0,NaN,NaN,2.0,1.0,1639.0,Central A/C,Average,Half Aluminum/Half Brick,Average 10,SPRINGFIELD,FRANCONIA,1.186381e+07,6.971774e+06,1.0,"Single-family, Detached"
702,0804 05190005,2023-03-13,620000.0,1961.0,NaN,NaN,2.0,NaN,1118.0,Central A/C,Average,Brick,Average 10,SPRINGFIELD,FRANCONIA,1.186121e+07,6.972030e+06,1.0,"Single-family, Detached"
896,0484 33 0027,2023-04-03,2181952.0,2023.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"Single-family, Detached"
905,0213 07 0030A,2023-07-31,1850000.0,1700.0,NaN,NaN,NaN,NaN,32.0,NaN,NaN,NaN,NaN,MCLEAN,DRANESVILLE,1.185188e+07,7.031923e+06,1.0,"Single-family, Detached"
1148,0203 03 0033,2023-08-28,1790000.0,1700.0,NaN,NaN,1.0,NaN,32.0,NaN,NaN,NaN,NaN,MCLEAN,DRANESVILLE,1.184010e+07,7.031498e+06,1.0,"Single-family, Detached"


In [231]:
prepared_df = prepared_df[
    ~(
        prepared_df['PRICE'].isna()
        | prepared_df['YRBLT'].isna() 
        | prepared_df['RMBED'].isna()
        | prepared_df['FIXBATH'].isna()
        | prepared_df['SFLA'].isna()
        | prepared_df['Supervisor District'].isna()
    )
]

Now let's use the WMATA station location data to get distance to different metrorail stations as an additional feature for each sale. First we need to get the latitude and longitude for each address, then we can calculate the distances from these addresses to wmata metrorail stations 

In [232]:
transformer = pyproj.Transformer.from_crs("epsg:2283", "epsg:4326", always_xy=True)
geod = pyproj.Geod(ellps="WGS84")
meters_in_mile = 1609.34

sale_locations_df = prepared_df[['PARID', 'Address Location X', 'Address Location Y']].drop_duplicates()
sale_locations_df[['address_lon', 'address_lat']] = (
    sale_locations_df
    .apply(lambda x: transformer.transform(x['Address Location X'], x['Address Location Y']), axis=1)
    .apply(pd.Series)
)

# cross join with stations data to get one record per address and station
wmata_stations_df = pd.read_csv("data/wmata_stations.csv")
# Filter to stations in Virginia, distance to stations across the river is likely not relevant
wmata_stations_df = wmata_stations_df[wmata_stations_df['station_state'] == 'VA']
sale_locations_df = sale_locations_df.merge(wmata_stations_df[['station_name', 'station_lat', 'station_lon']], how='cross')
sale_locations_df['distance'] = (
    sale_locations_df
    .apply(lambda x: geod.inv(x['address_lon'], x['address_lat'], x['station_lon'], x['station_lat'])[2] / meters_in_mile, axis=1)
)

# get the closest station and distance to closest station for each parcel
closest_station_df = (
    sale_locations_df.loc[sale_locations_df.groupby('PARID')['distance'].idxmin()]
    .reset_index()[['PARID', 'station_name', 'distance']]
    .rename(columns={'station_name':'closest_station', 'distance':'closest_station_dist'})
)

# pivot to get one record per parcel with distance to each station
station_dists_df = (
    sale_locations_df[['PARID', 'station_name', 'distance']]
    .pivot(index=['PARID'], columns=['station_name'], values=['distance'])
    .reset_index()
)

station_dists_df.columns = [
    x[0] if x[0] == 'PARID' else '_'.join(x).strip('_').lower().replace(' ', '_').replace('-', '_').replace('/', '_').replace('distance', 'dist') for x in station_dists_df.columns
]

In [233]:
prepared_df = prepared_df.merge(closest_station_df, on=['PARID'], how='left').merge(station_dists_df, on=['PARID'], how='left')

In [235]:
# clean up column names
prepared_df.columns = [x.lower().replace(' ', '_') for x in prepared_df.columns]

In [239]:
prepared_df.to_csv("data/sales_and_features_2023.csv", index=False)